In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from sys import stdout
import os, glob
from scipy.signal import detrend
from scipy import arctan2
import functions as f

In [3]:
import matplotlib.pyplot as plt

In [4]:
path_to_csvs   = "/Volumes/pdoubraw/repos/jet/data/TTU 200 m tower data/"
path_to_csvs   = "C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\"
filepaths_dict = f.get_file_paths(path_to_csvs, day=17)

In [5]:
height_feet_dict, height_meters_dict, header_names_dict, unique_heights, unique_heights_all = f.get_file_headers(path_to_csvs)

In [6]:
year   = 2017 
month  = 2
day    = 17
hour   = 12

desired_start = pd.Timestamp(year=year,month=month,day=day,hour=hour,minute=20)
desired_end   = desired_start + pd.Timedelta('10min')
timestamp_in_filename = desired_start.floor('30min')

In [7]:
filenames = sorted(glob.glob(os.path.join(path_to_csvs,'*{0:%Y%m%d}_T{0:%H%M}*'.format(timestamp_in_filename))))

In [8]:
filenames

['C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\FT2_E07_C04_R27880_D20170217_T1200_TR_D01.csv',
 'C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\FT2_E07_C04_R27880_D20170217_T1200_TR_D02.csv',
 'C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\FT2_E07_C04_R27880_D20170217_T1200_TR_D03.csv',
 'C:\\Users\\krydalch\\Desktop\\MetMastData\\TTU tall tower\\FT2_E07_C04_R27880_D20170217_T1200_TR_D04.csv']

In [9]:
perturbations = {}

for filepath in filenames:

    group = int(os.path.split(filepath)[-1].split('TR_D')[-1].split('.csv')[0])

    print(filepath)
        
    df           = f.csv_to_df(filepath, header_names_dict[group])  
    subdf        = f.subset_df(df)
    df_dict      = f.df_to_df_dict(subdf)

    df_dict_rot  = f.rotate_df_dict(df_dict)
    df_dict_pert = f.compute_perturbations(df_dict_rot)
    
    for height in df_dict_pert.keys():

        perturbations[height] = df_dict_pert[height].loc[desired_start:desired_end].copy()

C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27880_D20170217_T1200_TR_D01.csv
C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27880_D20170217_T1200_TR_D02.csv
C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27880_D20170217_T1200_TR_D03.csv
C:\Users\krydalch\Desktop\MetMastData\TTU tall tower\FT2_E07_C04_R27880_D20170217_T1200_TR_D04.csv


In [10]:
desired_index = pd.DatetimeIndex(start=desired_start, end=desired_end, freq='20ms').tz_localize('UTC')

C:\Users\krydalch\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  """Entry point for launching an IPython kernel.


In [11]:
for height in perturbations.keys():
    perturbations[height] = perturbations[height].reindex(desired_index).interpolate(method='index').iloc[0:30000].copy()

In [12]:
timescale = {}

for height in perturbations.keys():

    print(height)
    
    u     = perturbations[height].U
    uvals = u.values
    uidx  = u.index
    
    # the 10 segments are accessed by e.g., u_segments[isegment,:]
    segments = np.reshape(uvals,[10,3000])

    for iseg in range(10):
        series = pd.Series(segments[iseg,:])    
        rt = np.array([ series.autocorr(i) for i in range(3000) ])
        if iseg==0:
            r = rt.copy()
        else:
            r = r + rt    
    autocorr = r/float(iseg+1)

    firstNegIdx = (np.where(autocorr<0))[0][0]
    y = autocorr[0:firstNegIdx]
    x = (uidx-uidx[0]).total_seconds()[0:firstNegIdx]
    timescale[height] = np.trapz(y, x)

3.0


C:\Users\krydalch\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\krydalch\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\krydalch\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in less


8.0
13.0
33.0
55.0
155.0
382.0
519.0
656.0


In [13]:
mean_u = {}

for height in perturbations.keys():
    b = height*0.3048
    print(b ) 
    mean_u[height]     = perturbations[height].U.mean()

0.9144000000000001
2.4384
3.9624
10.0584
16.764
47.244
116.43360000000001
158.1912
199.9488


In [14]:
(pd.Series(timescale) * pd.Series(mean_u))

3.0        8.101208
8.0        8.714289
13.0       7.980271
33.0       9.770049
55.0      11.786363
155.0     13.399621
382.0     30.976991
519.0    111.210926
656.0     67.388044
dtype: float64